## Proceso de limpieza y normalización de datset IMDB Movie Reviews

Para lograr implementar cualquier modelo con éxito y resolver un problema, es importante realizar un preprocesamiento a los datos que se vayan a usar. Este proceso siempre se debe emplear porque los datos pueden estar sucios, contener ruido, datos faltantes, caracteres especiales, signos de puntuación, entre otras, los cuales no permitirán que el modelo sea entrenado correctamente. Por tal motivo, a continuación se efectúa el proceso de verificación, limpieza, transformación y división del conjunto para entrenamiento y pruebas

In [27]:
#Cargar librerias
import pandas as pd
import numpy as np
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re,string,unicodedata

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

import warnings
warnings.filterwarnings('ignore')


El conjunto de datos que utilizara para el proceso de análisis de sentimientos en reseñas de películas, está conformado por 50 mil datos. El conjunto de datos está constituido por dos columnas (review, sentiment). Para cargar los datos, se utiliza la librería Pandas y se muestra los 10 primeros datos que contiene el datset. La columna review contiene información con datos sucios y ruidos. 

In [3]:
#Muestra los 10 primeros review
data=pd.read_csv('IMDB Dataset.csv')
print(data.shape)
data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
#Imprime el review que se encuentra en la posición 4 
data['review'][4]

'Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit

### Verificar y limpiar de datos

Una vez cargados los datos se verifica que el dataset no contenga valores nulos. Y para esto, se utiliza la función isnull para la comprobación. Esta función devolverá "True" si encuentra componentes faltantes y "False" para los componentes que no faltan. Finalmente, se contará todos los valores "True" y como se puede observar, el conteo dio 0, es decir, no existe valores nulos en todo el dataset. 

In [5]:
#Se verifica si existe valores nullos en el dataset
data.isnull().sum()

review       0
sentiment    0
dtype: int64

En las siguientes líneas de código se realiza el proceso de limpieza de los datos. Se desarrolla una función para eliminar cualquier elemento HTML que exista en los review. Otra función para eliminar expresiones regulares, como lo son corchetes, llaves, asteriscos, comillas, entre otros. También se elimina texto ruidoso y finalmente, la eliminación de caracteres especiales. 

In [6]:
#Eliminar etiquetas de HTML 
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Eliminar algunas expresiones
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Elimina texto ruidoso
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Aplicar la función denoise_text en la columna de review
data['review']=data['review'].apply(denoise_text)

#---------------------------------------------

#Remueve caracteres especiales
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Aplicar la función remove_special_characters en la columna review
data['review']=data['review'].apply(remove_special_characters)

In [7]:
#Se verifica que los datos esten limpios
data['review'][4]

'Petter Matteis Love in the Time of Money is a visually stunning film to watch Mr Mattei offers us a vivid portrait about human relations This is a movie that seems to be telling us what money power and success do to people in the different situations we encounter This being a variation on the Arthur Schnitzlers play about the same theme the director transfers the action to the present time New York where all these different characters meet and connect Each one is connected in one way or another to the next person but no one seems to know the previous point of contact Stylishly the film has a sophisticated luxurious look We are taken to see how these people live and the world they live in their own habitatThe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits A big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterThe acting i

### Tokenizar

La tokenización sirve para encontrar y comprender patrones en el texto, el cual es útil para realizar el proceso de análisis de sentimiento. Este proceso es base para derivación y lematización del texto. Por tal motivo, en la siguiente línea de código se emplea el token y se configura las palabras vacías en el idioma en inglés.

In [8]:
#Tokenización de texto
tokenizer=ToktokTokenizer()
#Configuración de palabras vacías en inglés
stopword_list=nltk.corpus.stopwords.words('english')

### Stemming 

La implementación del Stemming se encarga de reducir las palabras a su raíz. Y es útil para asignar varias palabras a una palabra base en específico, esto no solo se hace con palabras, sino también con oraciones. Por tal motivo, en la siguiente línea de código se emplea Steamming en el texto.  

In [9]:
#Stemming del texto
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Aplicar la función en la columna review 
data['review']=data['review'].apply(simple_stemmer)

### Eliminar palabras varias 

Con el proceso de tokenización, se desarrolla una función que elimina todas las palabras vacias y se aplica el metodo en la columna review. 

In [46]:
#establecer palabras vacías en inglés
stop=set(stopwords.words('english'))
print(stop)

#eliminando las palabras vacías 
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#Aplicar la función 
data['review']=data['review'].apply(remove_stopwords)

{'between', 'll', 'don', 'weren', "aren't", 'herself', 'is', 'in', 'further', 'been', 'having', 'did', 'to', 'during', 'their', 'they', 'should', 'he', 'her', "don't", "haven't", 'be', 'the', 'out', "isn't", "weren't", 'few', 'yours', 'do', "it's", "didn't", 'you', 'mustn', 'by', 'will', 'these', 'has', 'each', 'them', 'only', 'under', 'yourselves', 'just', 'than', 'against', 'because', 'there', 'ain', 'who', 'my', 'through', 'being', 'ours', 'an', 'of', 'about', 'again', "shan't", 'more', "you'll", 'does', 'theirs', 'some', 'can', 'most', 'but', "mightn't", 'all', 'mightn', 'over', 'y', 'couldn', 'hasn', 'isn', 'o', 'am', 'until', 'whom', 'as', 'themselves', 'off', "you'd", 'ourselves', 'aren', "you're", 'why', 'hadn', 'wasn', 'his', 'this', 'which', 'that', 'such', 'not', 'haven', 'here', 'on', "won't", "she's", 's', 'me', "needn't", 'and', 'itself', 'are', "that'll", 'at', 'now', 'while', 'nor', 'we', "couldn't", 'up', 'when', 'doing', 'your', 'below', 'our', 'won', 'wouldn', 'own',

### Normalización 

Una vez terminado el proceso de limpieza de los datos, se emplea la normalización, es decir, estandariza todos los datos a un solo nive (palabras en minusculas) 

In [11]:

#nornalizar los reviews 
norm_reviews=data.review
norm_reviews[0]


'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

### Modelo Bags of words


Para realizar análisis de sentimiento se necesita crear una bolsa de palabras. El modelo Bag of Words es utilizado para representar los datos de prueba. Este método es empleado para PLN y IR. Para implementar este modelo se genera un vector de bolsas de palabras y después se hace un ajuste de los datos. 

In [34]:
#Contar vectorizador para bolsa de palabras
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

#Transformar datos
cv_reviews=cv.fit_transform(data.review)

print('BOW_cv_train:',cv_reviews.shape)

BOW_cv_train: (50000, 7528779)


### Transformación del etiquetado a datos binarios 

Una vez ya procesado los datos pertenecientes a la columna review, ahora solo falta la columna sentiment. En el caso de este dataset, la columna sentiment únicamente tiene dos valores: positive, negative. Y como tal, estos valores se los pueden convertir en números binarios, en donde el 0 representa a los negativos y el 1 a los positivos. La función LabelBinariser hace este proceso de binarización. 

In [35]:
#Función para convertir en binarios
lb=LabelBinarizer()

#transformación de los datos 
sentiment_data=lb.fit_transform(data['sentiment'])

print(sentiment_data.shape)

(50000, 1)


Ya terminada la limpieza y transformación de los datos, se procede a dividir el conjunto en datos de entrenamiento y datos para las pruebas. El conjunto de entrenamiento contiene 35 000 revisiones (70 % de los datos) destinadas al entrenamiento y el conjunto de prueba tiene 15 000 revisiones (30 % de los datos) para probar el clasificador.

### Dividir el conjunto

In [47]:
#Dividir el dataset en datos de entrenamiento y prueba 
X_train_reviews = cv_reviews[:35000]
Y_train_sentiments = sentiment_data[:35000]

X_test_reviews = cv_reviews[35000:]
Y_test_sentiments = sentiment_data[35000:]


El proceso de clasificación de características no se lo realiza porque el dataset solamente tiene una sola característica.